In [1]:
import pandas as pd

In [39]:
socios_tabla = pd.read_excel("../data/ICA_esqueleto.xlsx",sheet_name="R1.Principales países",skiprows=0)
socios_tabla.drop(1,inplace=True)
header = [socios_tabla.iloc[0].values, ["Exportación","Exportación","Exportación","Exportación",
                                        "Importación","Importación","Importación","Importación",
                                        "Saldo","Saldo"]]

header = ["<"+sup+" - "+inf+">" for inf, sup in zip(header[0],header[1])]
socios_tabla.columns = header

In [63]:
expo_socios = socios_tabla[16:].reset_index(drop=True)
expo_socios = expo_socios[expo_socios.columns[:4]] 
expo_socios.columns = ["País", "Exportación", "Variación porcentual","Participación porcentual"]
impo_socios = socios_tabla[16:].reset_index(drop=True)
impo_socios = impo_socios[impo_socios.columns[4:8]] 
impo_socios.columns = ["País", "Importación", "Variación porcentual","Participación porcentual"]
impo_socios

,País,Importación,Variación porcentual,Participación porcentual
0,China,14858,37,21
1,Brasil,13806,36.3,19.5
2,Estados Unidos,9219,91.3,13
3,Alemania,2345,11,3.3
4,Bolivia,2014,128.9,2.8
5,Paraguay,1775,-31.2,2.5
6,India,1655,42.1,2.3
7,Tailandia,1534,21.5,2.2
8,México,1424,29.1,2
9,Italia,1370,26.3,1.9


In [67]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#Separar más los títulos
#Hacer la opción para que sea gráfico único

specs = [[{'type':'domain'}, {'type':'domain'}]]
# cafe_colors =  ["#9e6e79","#798b92","#54a8ac","#2fc5c5","#09e1de","#43c1bd","#7ea19c","#b8817b","#d5716a","#f26059"]
fig = make_subplots(1, 2, specs=specs, subplot_titles=['Expo', 'Impo'])
fig.add_trace(go.Pie(labels=expo_socios.País, values= expo_socios.Exportación, scalegroup='one',), 1, 1)
fig.add_trace(go.Pie(labels=impo_socios.País, values= impo_socios.Importación, scalegroup='one',), 1, 2)

In [ ]:
# specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
# #Coloretes
# # cafe_colors =  ["#9e6e79","#798b92","#54a8ac","#2fc5c5","#09e1de","#43c1bd","#7ea19c","#b8817b","#d5716a","#f26059"]
# paises = list(participacion["Pais"].unique())
# # dic_color = {k:v for k,v in zip(paises, cafe_colors)}
# # participacion["color"]= participacion["Pais"].apply(lambda x: dic_color.get(x))

# fig = make_subplots(rows=2, cols=2, specs=specs, vertical_spacing=0.1,
#     subplot_titles=("Aceite de soja", "Harina y Pellets de soja", "Maíz","Poroto de soja"))

# fig.add_trace(go.Pie(labels=maiz["Pais"], values=maiz["Proporcion"]), 2,1)
# fig.add_trace(go.Pie(labels=harina_soja["Pais"], values=harina_soja["Proporcion"]), 1,2)
# fig.add_trace(go.Pie(labels=aceite_soja["Pais"], values=aceite_soja["Proporcion"]), 1,1)
# fig.add_trace(go.Pie(labels=poroto_soja["Pais"], values=poroto_soja["Proporcion"]), 2, 2)
# fig.update_layout(height=500, width=700, separators=",.", font_family="Georgia", margin ={'t': 20}
#                 #   , colorway=cafe_colors
#                   )
# # fig.marker.colors=cafe_colors
# note = 'Fuente: COMTRADE'
# fig.add_annotation(showarrow=False, text=note,font=dict(size=13), xref='paper',x=0.25,yref='paper', y=-0.07, 
#                            xanchor='right', yanchor='auto', xshift=0, yshift=0)
# # fig.update_traces(marker=dict(participacion["color"]))

# fig.show()